# Sistema de Triaje Multi-Agente para Tickets Técnicos

## Descripción

En este ejercicio, diseñarás un **sistema multi-agente que automatiza el triaje de tickets de soporte técnico** para operaciones aeroespaciales. Un agente coordinador (Triage Agent) orquestará tres agentes especializados que, trabajando en paralelo, evaluarán:

1. **Prioridad** (Critical, High, Medium, Low)
2. **Asignación de departamento** (Ingeniería, Manufactura, Control de Calidad, Cadena de Suministro, Mantenimiento)
3. **Estimación de esfuerzo** (Small, Medium, Large, Extra Large)

### Arquitectura
```
Ticket técnico
    ↓
[Triage Coordinator Agent]
    ├→ [Priority Agent] → Evalúa urgencia
    ├→ [Team Agent] → Asigna departamento
    └→ [Effort Agent] → Estima recursos
    ↓
Resumen integral de triaje para equipos
```

### Conceptos Clave
- **Connected Agent Tools**: Permite que agentes invoquen otros agentes
- **Multi-agente Architecture**: Escalable y reutilizable para diferentes tipos de tickets
- **Contexto de seguridad aeroespacial**: Evaluación basada en impacto en aeronavegabilidad

### Cargar librerías y variables de entorno

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ConnectedAgentTool, MessageRole, ListSortOrder
from azure.identity import DefaultAzureCredential

load_dotenv(find_dotenv(usecwd=True))
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")

### Conectar al Cliente de Agentes (Agent Client)

In [ ]:
print("Connecting to Azure AI Agents Client...")
agents_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True,
        exclude_managed_identity_credential=True,
    )
)
print("Agents client connected successfully")

### Crear Agentes Especializados

Primero, creamos los tres agentes especializados: uno para priorizar tickets, otro para asignar equipos y un tercero para estimar el esfuerzo.

In [ ]:
# Priority evaluation agent
priority_agent_name = "priority_agent"
priority_agent_instructions = """
Evalúa la urgencia de un ticket técnico para operaciones aeroespaciales según su descripción.

    Responde con uno de los siguientes niveles:
    - Critical: Problemas en componentes críticos de seguridad, paro de línea de producción, o fallos de calidad que afecten la aeronavegabilidad
    - High: Retrasos de producción, defectos que impactan plazos de entrega, o problemas con impacto en clientes
    - Medium: Incidencias gestionables pero sensibles al tiempo, preocupaciones menores de calidad, o necesidades de optimización de procesos
    - Low: Actualizaciones de documentación, mejoras no urgentes, o cuestiones cosméticas

Devuelve solo el nivel de urgencia y una explicación muy breve enfocada en el impacto aeroespacial.
"""
print("Creating priority agent...")
priority_agent = agents_client.create_agent(
    model=model_deployment,
    name=priority_agent_name,
    instructions=priority_agent_instructions,
)
print(f"Priority agent created (id: {priority_agent.id})")

# Team assignment agent
team_agent_name = "team_agent"
team_agent_instructions = """
Decide qué departamento debe gestionar cada ticket técnico.

    Elige entre los siguientes departamentos:
    - Engineering: Problemas de diseño, especificaciones de componentes, análisis de rendimiento, I+D
    - Manufacturing: Procesos de producción, incidencias en líneas de ensamblaje, utillaje, fabricación
    - Quality Control: Inspecciones, pruebas, no conformidades, certificaciones, cumplimiento de normas
    - Supply Chain: Aprovisionamiento de materiales, búsqueda de componentes, inventario, logística
    - Maintenance: Mantenimiento de equipos, infraestructura de planta, mantenimiento preventivo

Basa tu respuesta en el contenido técnico del ticket. Devuelve el nombre del departamento y una explicación muy breve.
"""
print("Creating team assignment agent...")
team_agent = agents_client.create_agent(
    model=model_deployment,
    name=team_agent_name,
    instructions=team_agent_instructions,
)
print(f"Team assignment agent created (id: {team_agent.id})")

# Effort estimation agent
effort_agent_name = "effort_agent"
effort_agent_instructions = """
Estima el esfuerzo de ingeniería requerido para cada ticket técnico aeroespacial.

    Usa la siguiente escala:
    - Small: Se resuelve en 1-2 días (ajustes menores, documentación, correcciones simples)
    - Medium: 3-5 días de trabajo (análisis de componentes, ajustes de proceso, pruebas estándar)
    - Large: 1-2 semanas (análisis de causa raíz, rediseño, requisitos de certificación)
    - Extra Large: Varias semanas o esfuerzo transversal (rediseño mayor, cumplimiento regulatorio, coordinación con cliente)

Considera factores de complejidad aeroespacial: requisitos de certificación, criticidad de seguridad e impacto en producción.
Devuelve el nivel de esfuerzo y una breve justificación técnica.
"""
print("Creating effort estimation agent...")
effort_agent = agents_client.create_agent(
    model=model_deployment,
    name=effort_agent_name,
    instructions=effort_agent_instructions,
)
print(f"Effort estimation agent created (id: {effort_agent.id})")

### Crear Herramientas de Agentes Conectados (Connected Agent Tools)

Para que el agente coordinador pueda invocar a los agentes especializados, creamos herramientas para cada uno.

In [ ]:
print("Creating connected agent tools...")
priority_agent_tool = ConnectedAgentTool(
    id=priority_agent.id, name=priority_agent_name, description="Evaluate ticket priority"
)
team_agent_tool = ConnectedAgentTool(
    id=team_agent.id, name=team_agent_name, description="Determine which team should handle ticket"
)
effort_agent_tool = ConnectedAgentTool(
    id=effort_agent.id, name=effort_agent_name, description="Estimate effort required to complete ticket"
)
print("Connected agent tools created")

### Crear el Agente Coordinador de Triaje

Este agente orquestará el trabajo de los agentes especializados.

In [ ]:
triage_agent_name = "triage-agent"
triage_agent_instructions = """
Eres el coordinador de triaje técnico para operaciones aeroespaciales.
Analiza cada ticket técnico entrante y usa los agentes especialistas conectados para determinar:
    1. Nivel de prioridad según seguridad aeroespacial e impacto en producción
    2. Asignación al departamento adecuado
    3. Estimación del esfuerzo de ingeniería considerando la complejidad aeroespacial

Proporciona un resumen de triaje integral para equipos de fabricación e ingeniería.
"""
print("Creating triage coordinator agent...")
triage_agent = agents_client.create_agent(
    model=model_deployment,
    name=triage_agent_name,
    instructions=triage_agent_instructions,
    tools=[
        priority_agent_tool.definitions[0],
        team_agent_tool.definitions[0],
        effort_agent_tool.definitions[0],
    ],
)
print(f"Triage coordinator agent created (id: {triage_agent.id})")

### Usar los agentes para el triaje de un ticket

Introduce un problema de soporte en el `prompt` y ejecuta la celda para que los agentes lo procesen.

In [ ]:
print("Creating thread...")
thread = agents_client.threads.create()
print(f"Thread created (id: {thread.id})")

# Enter support issue here
prompt = "Se detectaron tolerancias dimensionales fuera de especificación en álabes de turbina de alta presión. Lote Q1-2024-TB-156 afectado. 23 unidades potencialmente no conformes requieren inspección metalúrgica completa antes de entrega a cliente."

print(f"Sending prompt: '{prompt}'")
message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=prompt,
)

print("\nProcessing triage... please wait.")
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=triage_agent.id)
print(f"Triage run completed with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    print("Retrieving triage results...")
    messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for message in messages:
        if message.text_messages:
            last_msg = message.text_messages[-1]
            print(f"{message.role}:\n{last_msg.text.value}\n")

### Limpieza

Finalmente, eliminamos todos los agentes creados.

In [ ]:
print("Cleaning up agents...")
print(f"Deleting triage agent (id: {triage_agent.id})...")
agents_client.delete_agent(triage_agent.id)
print("Triage agent deleted")

print(f"Deleting priority agent (id: {priority_agent.id})...")
agents_client.delete_agent(priority_agent.id)
print("Priority agent deleted")

print(f"Deleting team assignment agent (id: {team_agent.id})...")
agents_client.delete_agent(team_agent.id)
print("Team assignment agent deleted")

print(f"Deleting effort estimation agent (id: {effort_agent.id})...")
agents_client.delete_agent(effort_agent.id)
print("Effort estimation agent deleted")

print("All agents cleaned up successfully")